## Tensorflow四 RNN/LSTM基础之name/variable_scope
#### @autor: cs
#### @date: 2017-08-18

In [1]:
#参考 refer to: https://www.tensorflow.org/programmers_guide/variable_scope 

### 1.实验1.三种方式创建变量： tf.placeholder, tf.Variable, tf.get_variable

In [1]:
# 探索三种方式定义的变量之间的区别。
import tensorflow as tf
# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = False
sess = tf.Session(config=config)

In [2]:
# 1.placeholder 
v1 = tf.placeholder(tf.float32, shape=[2,3,4])
print(v1.name)
v1 = tf.placeholder(tf.float32, shape=[2,3,4], name='ph')
print(v1.name)
v1 = tf.placeholder(tf.float32, shape=[2,3,4], name='ph')
print(v1.name)
# 属于类tensorflow.python.framework.ops.Tensor
print(type(v1))
print(v1)

Placeholder:0
ph:0
ph_1:0
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("ph_1:0", shape=(2, 3, 4), dtype=float32)


In [3]:
# 2. tf.Variable()
v2 = tf.Variable([1,2], dtype=tf.float32)
print(v2.name)
v2 = tf.Variable([1,2], dtype=tf.float32, name='V')
print(v2.name)
v2 = tf.Variable([1,2], dtype=tf.float32, name='V')
print(v2.name)
# 属于tensorflow.python.ops.variables.Variable
print(type(v2))
print(v2)

Variable:0
V:0
V_1:0
<class 'tensorflow.python.ops.variables.Variable'>
<tf.Variable 'V_1:0' shape=(2,) dtype=float32_ref>


In [4]:
# 3.tf.get_variable() 创建变量的时候必须要提供 name
v3 = tf.get_variable(name='gv', shape=[])  
print(v3.name)
v4 = tf.get_variable(name='gv', shape=[2])
print(v4.name)
# 报错，原因是命名冲突

gv:0


ValueError: Variable gv already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "H:\Software\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1204, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "H:\Software\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 2630, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "H:\Software\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [5]:
print(type(v3))
print(v3)

<class 'tensorflow.python.ops.variables.Variable'>
<tf.Variable 'gv:0' shape=() dtype=float32_ref>


In [7]:
# tf.trainable_variables(), 它能够将我们定义的所有的 trainable=True 的所有变量以一个list的形式返回。
# 利用这个函数，我们可以确定那些变量在定义时是trainable=True的
vs = tf.trainable_variables()
print(len(vs))
for v in vs:
    print(v)

4
<tf.Variable 'Variable:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'V:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'V_1:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'gv:0' shape=() dtype=float32_ref>


In [8]:
# 以上说明tf.placeholder在定义变量时默认trainable=False, tf.Variable, tf.get_variable是默认rainable=True的

In [9]:
# tf.Variable 和 tf.get_variable之间 不会发生命名冲突, 如果定义时冲突了，会自动更改。
# 例如
var1 = tf.Variable([1,2], dtype=tf.float32, name='var1')
print('var1.name: ', var1.name)
var2 = tf.get_variable(name='var1', shape=[])
print('var2.name: ', var2.name)
print(var1)
print(var2)

var1.name:  var1:0
var2.name:  var1_1:0
<tf.Variable 'var1:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'var1_1:0' shape=() dtype=float32_ref>


In [10]:
vs = tf.trainable_variables()
print(len(vs))
for v in vs:
    print(v)

6
<tf.Variable 'Variable:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'V:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'V_1:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'gv:0' shape=() dtype=float32_ref>
<tf.Variable 'var1:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'var1_1:0' shape=() dtype=float32_ref>


In [12]:
sess.close()

### 实验2. 探索 name_scope 和 variable_scope

In [13]:
# 配置
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = False
sess = tf.Session(config=config)

In [17]:
with tf.name_scope('nsc1'):
    v1 = tf.Variable([1], name='v1')
    with tf.variable_scope('vsc1'):
        v2 = tf.Variable([1], name='v2')
        vh = tf.get_variable(name='vh', shape=[])
print('v1.name: ', v1.name)
print('v2.name: ', v2.name)
print('vh.name: ', vh.name)
sess.close()

v1.name:  nsc1_2/v1:0
v2.name:  nsc1_2/vsc1/v2:0
vh.name:  vsc1/vh:0


### 2.第一组，用 tf.Variable() 的方式来定义。

In [23]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [30]:
%clear

In [31]:
import gc
gc.collect()

928

In [1]:
import tensorflow as tf
# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# 拿官方的例子改动一下
def my_image_filter():
    conv1_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv1_weights")
    conv1_biases = tf.Variable(tf.zeros([32]), name="conv1_biases")
    conv2_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv2_weights")
    conv2_biases = tf.Variable(tf.zeros([32]), name="conv2_biases")
    return None

# First call creates one set of 4 variables.
result1 = my_image_filter()
# Another set of 4 variables is created in the second call.
result2 = my_image_filter()
# 获取所有的可训练变量
vs = tf.trainable_variables()
print('There are %d train_able_variables in the Graph: ' % len(vs))
for v in vs:
    print(v)

There are 8 train_able_variables in the Graph: 
<tf.Variable 'conv1_weights:0' shape=(5, 5, 32, 32) dtype=float32_ref>
<tf.Variable 'conv1_biases:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'conv2_weights:0' shape=(5, 5, 32, 32) dtype=float32_ref>
<tf.Variable 'conv2_biases:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'conv1_weights_1:0' shape=(5, 5, 32, 32) dtype=float32_ref>
<tf.Variable 'conv1_biases_1:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'conv2_weights_1:0' shape=(5, 5, 32, 32) dtype=float32_ref>
<tf.Variable 'conv2_biases_1:0' shape=(32,) dtype=float32_ref>


### 2.第二种方式，用 tf.get_variable() 的方式

In [1]:
import tensorflow as tf
# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# 下面是定义一个卷积层的通用方式, 此处使用tf.get_variable和下面的tf.variable_scop搭配使用实现变量共享
def conv_relu(kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    return None


def my_image_filter():
    # 按照下面的方式定义卷积层，非常直观，而且富有层次感
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu([5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu( [5, 5, 32, 32], [32])

    
with tf.variable_scope("image_filters") as scope:
    # 下面我们两次调用 my_image_filter 函数，但是由于引入了 变量共享机制
    # 可以看到我们只是创建了一遍网络结构。
    result1 = my_image_filter()
    scope.reuse_variables()
    result2 = my_image_filter()


# 看看下面，完美地实现了变量共享！！！
vs = tf.trainable_variables()
print('There are %d train_able_variables in the Graph: ' % len(vs))
for v in vs:
    print(v)

There are 4 train_able_variables in the Graph: 
<tf.Variable 'image_filters/conv1/weights:0' shape=(5, 5, 32, 32) dtype=float32_ref>
<tf.Variable 'image_filters/conv1/biases:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'image_filters/conv2/weights:0' shape=(5, 5, 32, 32) dtype=float32_ref>
<tf.Variable 'image_filters/conv2/biases:0' shape=(32,) dtype=float32_ref>


In [2]:
# 在深度学习中，通常说到 变量共享我们都会想到 RNN 。下面两个源码中非常漂亮的例子，可以参考学习学习。

In [5]:
# 例1：MultiRNNCell(RNNCell) 中这样来创建各层 Cell
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py

In [6]:
# 例2：tf.contrib.rnn.static_bidirectional_rnn 双端 LSTM 